# Predicting medical appointments using Python

In this notebook, I'll use Python and its libraries to predict whether someone would show up for a medical appointment or not. I'll develop an Artificial Neural Network and train my model on the data.

## Import libraries

I'll import the necessary deep learning libraries, Keras and Tensorflow along with some metrics.

In [1]:
import pandas as pd

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


## Import dataset

In [2]:
dataset = pd.read_csv('data/dataset_modified.csv')
dataset.head(5)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Showed_up,Date.diff
0,2.987250e+13,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,False,True,False,False,False,False,True,0
1,5.589978e+14,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,False,False,False,False,False,False,True,0
2,4.262962e+12,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,False,False,False,False,False,False,True,0
3,8.679512e+11,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,False,False,False,False,False,False,True,0
4,8.841186e+12,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,False,True,True,False,False,False,True,0


## Data engineering

If a person has missed an appointment before, there are chances he/she might miss again. Let's see if that is correlated.
I found this idea from a kernel on [Kaggle](https://www.kaggle.com/belagoesr/predicting-no-show-downsampling-approach-with-rf).

In [3]:
missed_appointment = dataset.groupby('PatientId')['Showed_up'].sum()
missed_appointment = missed_appointment.to_dict()
dataset['missed_appointment_before'] = dataset.PatientId.map(lambda x: 1 if missed_appointment[x]>0 else 0)
dataset['missed_appointment_before'].corr(dataset['Showed_up'])

0.6102086525921399

Surprisingly the correlation is really high and we should keep this column.

As we don't need all the columns, I'll start ommiting them.

In [4]:
dataset = dataset.drop(['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay'], axis = 1)
print("Columns: {}".format(dataset.columns))

Columns: Index(['Gender', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'Showed_up',
       'Date.diff', 'missed_appointment_before'],
      dtype='object')


Let's great dummy columns to accomodate all neighborhoods.

In [5]:
dataset = pd.concat([dataset.drop('Neighbourhood', axis = 1), 
           pd.get_dummies(dataset['Neighbourhood'])], axis=1)

Now, let's map the Gender column to random values, here 'M' as 0 and 'F' as 1.

In [6]:
gender_map = {'M': 0, 'F': 1}
dataset['Gender'] = dataset['Gender'].map(gender_map)

Next, I'll split the dataset into train and test data.

In [7]:
y = dataset.loc[:, 'Showed_up']
X = dataset.drop(['Showed_up'], axis = 1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [9]:
print("Final shape: {}".format(X_train.shape))

Final shape: (71681, 91)


Let's now scale the data to make it ready for the Neural Network.

In [10]:
standardScaler = StandardScaler()
X_train = standardScaler.fit_transform(X_train)
X_test = standardScaler.transform(X_test)

## Model generation

I'll develop an Artificial Neural Network to map the data to find patterns and eventually learn from it.

In [11]:
classifier = Sequential()
classifier.add(Dense(units = 64, activation = 'relu', input_dim = 91))
classifier.add(Dropout(rate = 0.5))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate = 0.5))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate = 0.5))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                5888      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                

In [12]:
history = classifier.fit(X_train, y_train, epochs = 5, validation_split = 0.1)

Train on 64512 samples, validate on 7169 samples
Epoch 1/5
64512/64512 [==============================] - 3s 44us/step - loss: 0.4022 - accuracy: 0.8625 - val_loss: 0.3564 - val_accuracy: 0.8824
Epoch 2/5
64512/64512 [==============================] - 3s 40us/step - loss: 0.3463 - accuracy: 0.8834 - val_loss: 0.3418 - val_accuracy: 0.8827
Epoch 3/5
64512/64512 [==============================] - 2s 39us/step - loss: 0.3404 - accuracy: 0.8838 - val_loss: 0.3419 - val_accuracy: 0.8827
Epoch 4/5
64512/64512 [==============================] - 3s 39us/step - loss: 0.3359 - accuracy: 0.8840 - val_loss: 0.3365 - val_accuracy: 0.8827
Epoch 5/5
64512/64512 [==============================] - 3s 42us/step - loss: 0.3335 - accuracy: 0.8839 - val_loss: 0.3340 - val_accuracy: 0.8827


## Model prediction

As the model is now ready and trained, let's test on the test data.
For a baseline, I'll also write the difference between the two classes in the test data.

In [13]:
y_pred = classifier.predict(X_test)
y_pred = y_pred > 0.5

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))
print("-"*50)
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred)*100))

Confusion matrix:
[[ 3032  4127]
 [    0 28147]]
--------------------------------------------------
Accuracy: 88.31%


## Conclusion

Using `ANN` with data engineering, I was able to achieve an accuracy of over 88% in predicting whether someone would show up or not for their appointment.